Import needed libraries.

- BeautifulSoup is used for web scrapping purposes

- requests, urllib, and ssl are used for making api request

- json used to deal with json format

- pandas used for dealing with data and exporting

Potential Additions: Extracting Keywords and Citations

- However citations just give another article ID.

In [1]:
from bs4 import BeautifulSoup
import requests, urllib, ssl, json
import pandas as pd

API Request Key

In [2]:
api_req_url="https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=json&retmax=100&sort=relevance&term="
keyword = "KEYWORD"

Ask for user input

In [3]:
keyword = str(input("Enter the keyword required: \n"))
api_req_url = api_req_url + keyword

Enter the keyword required: 
immport


Need to verify your HTTP Certificate and then make request.

In [4]:
_create_unverified_https_context = ssl._create_unverified_context
request = urllib.request.urlopen(api_req_url).read()

Now need to be able to access JSON data and parse specific things

TODO: Could add a feature for inputting how many articles wanted and also wanted/totalmatch ratio

In [5]:
data = json.loads(request)

article_ids = data['esearchresult']['idlist']
article_ids

['29485622',
 '32411773',
 '24791905',
 '31272390',
 '32504750',
 '28057685',
 '30852461',
 '30977847',
 '30304689',
 '32600423',
 '32375704',
 '32351547',
 '32310824',
 '32068352',
 '32355728',
 '32296685',
 '32214002',
 '32019546',
 '31921619',
 '31821170',
 '30832680',
 '30594555',
 '30516029',
 '30398470',
 '27377652',
 '27130330',
 '20839340',
 '26112029',
 '20460173',
 '32283555',
 '28851925',
 '25857935',
 '24862987',
 '24161793',
 '22959960',
 '20044059']

Main logic here.

Basically scrapes the API return and then gets key information including the abstract that can be used later

In [6]:
def get_elements(soup_object):
    
    authors = getAuthors(soup_object)
    title = getTitle(soup_object)
    abstract = getAbstract(soup_object)
    pubmed_id = getID(soup_object)
    
    journal_volume = ""
    journal_title = ""
    journal_date = ""
    
    if soup_object.find("JournalIssue"):
        if soup_object.find("JournalIssue").find("Volume"):
            journal_volume = soup_object.find("JournalIssue").find("Volume").text
    if soup_object.find("Title"):
        journal_title = soup_object.find("Title").text
    if soup_object.find("JournalIssue").find("Year") and soup_object.find("JournalIssue").find("Month") and soup_object.find("JournalIssue").find("Day"):
        journal_date = soup_object.find("JournalIssue").find("Year").text + " " + soup_object.find("JournalIssue").find("Month").text + " " + soup_object.find("JournalIssue").find("Day").text
    
    row_list = []
    row_list.append(authors)
    row_list.append(title)
    row_list.append(journal_title)
    row_list.append(journal_volume)
    row_list.append(journal_date)
    row_list.append(pubmed_id)
    row_list.append(abstract)
    
    return row_list
    
def getAuthors(soup_object):
    authors = ""
    authorlist = soup_object.find("AuthorList")
    ##print(authorlist)
    
    if authorlist: 
        for i in range(len(authorlist.find_all("LastName"))):
            firstName = authorlist.find_all("ForeName")[i].text
            authors = authors + firstName
            lastName = authorlist.find_all("LastName")[i].text
            authors = authors + " " + lastName
            
            if i != len(authorlist.find_all("LastName")) - 1:
                authors = authors + ", "
    print(authors)
    return authors

def getTitle(soup_object):
    title = ""
    if soup_object.find("AricleTitle"):
        title_element = soup_object.find("ArticleTitle")
        title = title_element.text

    print(title)
    return title
    
def getAbstract(soup_object):
    abstract = ""
    if soup_object.find("AbstractText"):
        abstract = soup_object.find("AbstractText").text
    print(abstract)
    return abstract

def getID(soup_object):
    idInfo = ""
    if soup_object.find("PMID"):
        idInfo = "PMID: "
        idInfo += soup_object.find("PMID").text
    return idInfo

Loops through all the different IDs and the scraping material that was gathered using the above logic

In [7]:
all_articles = []
for articleID in article_ids:
   
    get_url = "http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&retmode=xml&id="
    get_url = get_url + articleID
    
    _create_unverified_https_context = ssl._create_unverified_context
    response = requests.get(get_url)

    soup_object = BeautifulSoup(response.content, "xml")
    one_article = get_elements(soup_object)
    all_articles.append(one_article)

Sanchita Bhattacharya, Patrick Dunn, Cristel G Thomas, Barry Smith, Henry Schaefer, Jieming Chen, Zicheng Hu, Kelly A Zalocusky, Ravi D Shankar, Shai S Shen-Orr, Elizabeth Thomson, Jeffrey Wiser, Atul J Butte

Immunology researchers are beginning to explore the possibilities of reproducibility, reuse and secondary analyses of immunology data. Open-access datasets are being applied in the validation of the methods used in the original studies, leveraging studies for meta-analysis, or generating new hypotheses. To promote these goals, the ImmPort data repository was created for the broader research community to explore the wide spectrum of clinical and basic research data and associated findings. The ImmPort ecosystem consists of four components-Private Data, Shared Data, Data Analysis, and Resources-for data archiving, dissemination, analyses, and reuse. To date, more than 300 studies have been made freely available through the Shared Data portal (www.immport.org/immport-open), which al

Rongzhi Huang, Min Mao, Yunxin Lu, Qingliang Yu, Liang Liao

Melanoma is a cancer of the skin with potential to spread to other organs and is responsible for most deaths due to skin cancer. It is imperative to identify immune biomarkers for early melanoma diagnosis and treatment.
Xiao-Yan Sun, Shi-Zhe Yu, Hua-Peng Zhang, Jie Li, Wen-Zhi Guo, Shui-Jun Zhang

Hepatocellular carcinoma (HCC) has become the second most common tumor type that contributes to cancer-related death worldwide. The study aimed to establish a robust immune-related gene pair (IRGP) signature for predicting the prognosis of HCC patients.
Weijie Qiang, Yifei Dai, Guibo Sun, Xiaoyan Xing, Xiaobo Sun

Colon adenocarcinoma (COAD) is one of the most commonly diagnosed cancers, and it is closely related to the immune microenvironment. Considering that immunotherapy is not effective for all COAD patients, it is necessary to identify the effective population before administering treatment. In this study, we established an in

Qian Song, Jun Shang, Zuyi Yang, Lanlin Zhang, Chufan Zhang, Jianing Chen, Xianghua Wu

Lung cancer has become the most common cancer type and caused the most cancer deaths. Lung adenocarcinoma (LUAD) is one of the major type of lung cancer. This study aimed to establish a signature based on immune related genes that can predict patients' OS for LUAD.
Sipeng Shen, Guanrong Wang, Ruyang Zhang, Yang Zhao, Hao Yu, Yongyue Wei, Feng Chen

Ovarian cancer (OV) is the most lethal gynecological cancer in women. We aim to develop a generalized, individualized immune prognostic signature that can stratify and predict overall survival for ovarian cancer.
Zhongyu Wang, Qian Song, Zuyi Yang, Jianing Chen, Jun Shang, Wen Ju

The kidney renal papillary cell carcinoma (KIRP) is a relatively rare type of kidney cancer. There has been no investigation to find a robust signature to predict the survival outcome of KIRP patients in the aspect of tumor immunology. In this study, 285 KIRP samples from The Ca

Randi Vita, James A Overton, Patrick Dunn, Kei-Hoi Cheung, Steven H Kleinstein, Alessandro Sette, Bjoern Peters

An Immune Exposure is the process by which components of the immune system first encounter a potential trigger. The ability to describe consistently the details of the Immune Exposure process was needed for data resources responsible for housing scientific data related to the immune response. This need was met through the development of a structured model for Immune Exposures. This model was created during curation of the immunology literature, resulting in a robust model capable of meeting the requirements of such data. We present this model with the hope that overlapping projects will adopt and or contribute to this work.
Harry Pickering, Andy Teng, Nkoyo Faal, Hassan Joof, Pateh Makalo, Eunice Cassama, Meno Nabicassa, Anna R Last, Sarah E Burr, Sarah L Rowland-Jones, Nicholas R Thomson, Chrissy H Roberts, David C W Mabey, Robin L Bailey, Richard D Hayward, Luis M de la Ma

Converts the gathered data and exports it as a CSV file

In [8]:
data_frame = pd.DataFrame(all_articles)
data_frame.columns = ["Authors", "ArticleTitle", "JournalTitle", "JournalVolume", "JournalDate", "PUBID", "Abstact"]

csv_name = keyword + ".csv"
data_frame.to_csv(csv_name)